In [1]:
import pandas as pd
import os
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import tqdm

In [2]:
path = '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - statin/8. RNA-seq/Elizabeth/LW15 analysis/LW15_normalized.csv'
data = pd.read_csv(path)
# data.head(5)
gene_conversions = data[['gene_id','gene_name']]
gene_conversions

,gene_id,gene_name
0,ENSMUSG00000000001,Gnai3
1,ENSMUSG00000000028,Cdc45
2,ENSMUSG00000000031,H19
3,ENSMUSG00000000049,Apoh
4,ENSMUSG00000000056,Narf
...,...,...
24637,ENSMUSG00002076463,---
24638,ENSMUSG00002076508,---
24639,ENSMUSG00002076570,7SK
24640,ENSMUSG00002076650,---


In [3]:
data = data[['gene_id','RC.1','RC.2','RC.3','H1.1','H1.2','H1.3', 'H12.1','H12.2','H12.3','RVN.1','RVN.2','RVN.3','H2.1','H2.2','H2.3']]
data.set_index('gene_id', inplace=True)
data.head(5)

,RC.1,RC.2,RC.3,H1.1,H1.2,H1.3,H12.1,H12.2,H12.3,RVN.1,RVN.2,RVN.3,H2.1,H2.2,H2.3
gene_id,,,,,,,,,,,,,,,
ENSMUSG00000000001,231.612000,230.273000,155.366000,142.313000,203.244000,169.737000,134.572000,125.387000,152.344000,284.066000,242.846000,229.793000,139.686000,296.971000,279.698000
ENSMUSG00000000028,71.304500,67.238700,75.939800,76.444600,73.306700,81.362700,65.952100,56.257800,70.416500,65.270700,73.563000,75.933100,62.730700,78.365200,73.653300
ENSMUSG00000000031,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.971353,0.989254,0.952513,0.028285,0.000003,0.053162
ENSMUSG00000000049,0.060922,0.000003,0.026207,0.058292,0.000003,0.000003,0.023422,0.000003,0.000003,0.107931,0.239821,0.148833,0.000003,0.061660,0.079742
ENSMUSG00000000056,33.078700,32.345900,27.514600,15.388400,20.395100,18.795900,18.267600,14.648800,20.454900,33.457600,31.626100,31.313800,14.282500,22.504800,23.018000


In [ ]:
old_genes = {'Main 8 Genes':['Atp1a1','Cltc','Ifngr1','Gpx3','Srebf2','Alyref','Niban1','Mybbp1a'], 'SREBF':['Ldlr','Srebf1','Srebf2'], 'HMGCR':['Acat1','Acat2','Hmgcr','Mvd','Pmvk','Fdps','Ggps1'],'Hypoxia':['Hif1a','Vhl','Arnt','Epas1','Vegfa','Vegfb','Slc2a1','Vegfd'],'EMT':['Cdh1','Cdh2','Acta2','Mmp9','Snai2','Snai1','Zeb1','Ntn1','Zeb2'],'Cholesterol':['Acaca','Ldlr','Fasn','Scd1','Srebf1','Spring1'], 'Other 1':['Myc','Kras','Nras','Mki67','Casp3','Uchl1'], 'Other 2':['Agfg1','Rab11a','Rptor','Lrp1','Rac1','Rac2','Rock1','Rock2'], 'Other 3':['Txnrd1','Txnrd2','Mdm2','E2f1','Trp53','Pparg','Ppard','Prdx5','Mtor'], 'Other 4':['Usp7','Mdm4','Arf1','Calr','Trib3','Ddit3','Mapk1','Sqstm1','P4ha1']}

# convert the target genes to ensembl IDs
# Flatten the list of genes from target_genes dict
old_gene_list = [gene for category in old_genes.values() for gene in category]
# First, create a dictionary from the gene_conversions DataFrame
gene_name_to_ensembl_dict = pd.Series(gene_conversions['gene_id'].values,index=gene_conversions['gene_name']).to_dict()

# Now, map the old_gene_list to Ensembl IDs using this dictionary
old_gene_list = [gene_name_to_ensembl_dict[gene] for gene in old_gene_list if gene in gene_name_to_ensembl_dict]

excel_file_path = '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/Functional screening/240122_MI_320genes_list.xlsx'
new_genes = pd.read_excel(excel_file_path, sheet_name='Sheet2')
new_genes = new_genes[['Ensembl','found in']]
genes_dict = {}
categories = new_genes['found in'].unique()
# print(categories)
# add category keys to dict
for category in categories:
    genes_dict[category] = []

# Iterate through the rows of the DataFrame
for index, row in new_genes.iterrows():
    gene = row['Ensembl']
    category = row['found in']
    genes_dict[category].append(gene)

new_gene_list = new_genes['Ensembl']

In [9]:
id_to_name_dict = pd.Series(gene_conversions['gene_name'].values,index=gene_conversions['gene_id']).to_dict()
gene_names_dict = {}

for category, list in genes_dict.items():
    gene_names_dict[category] = []
    for gene_id in list:
        if gene_id in id_to_name_dict:
            gene_name = id_to_name_dict.get(gene_id)
        else:
            gene_name = gene_id    
            print(gene_id)
        gene_names_dict[category].append(gene_name)
        

In [8]:
print(gene_names_dict)

{'PCA': ['4931428F04Rik', 'Caskin1', 'Scarf2', 'Vps4a', 'Card10', 'Thap11', 'Nup50', 'Rrad', 'Eps8l2', 'Cited1', 'Reep6', 'Ccdc92', 'Trp53rka', 'Gfer', 'Shf', 'Stac2', 'Zfp395', 'Sptbn4', 'B4galt3', 'Sh3bp1', 'Rasgrp4', 'Spata2l', 'Arid3c', 'Vill', 'Rfng', 'Nlgn2', 'Syce2', 'Prph', 'Mkrn1', 'Gpr157', 'Foxa3', 'Zcwpw1', 'Prrx2', 'Ldhd', 'Stard10', 'Arsi', 'Plvap', 'Prss16', 'Fam117a', 'Trim62', 'Eva1c', 'Lrrc73', 'Tradd', 'Dact3', 'Ptpn6', 'Rtraf', 'Gm9747', 'Pgf', 'Mypop', 'Fbxo17', 'Hoxc11', 'Glo1', 'Grik5', 'Brsk1', 'Slc24a3', 'Tmem121', 'Plekhg6', 'Prss41', 'Crnde', 'Dpf1', 'Crlf1', 'Ptpn18', 'Abcg4', 'Adam11', 'Bcar1', 'Hoxc5', 'Adgrl1', 'Dhcr24', 'Dpf3', 'Batf3', 'Cbarp', 'Hic2', 'Mpzl1', 'Tpi1', 'Dlgap3', 'Gm10157', 'Ap1m2', 'Samd14', 'Map4k1', '2410004I01Rik', 'Crabp1', 'Fndc4', 'A430093F15Rik', 'Pcsk4', 'Fbxl14', 'Tcf15', 'Ninl', 'Rcor2', 'Nckipsd', 'Dusp2', 'Lrrc23', 'Cadm4', 'Ecd', 'Esrrb', 'Nrtn', 'Dnajc9', 'Cd164l2', 'Sema6b', 'Tmem151b', 'Irf6', 'Mdfi', '---', 'Mafa', 'Wnt

In [10]:
# Melting the dataframe to long format for ANOVA
melted_data = pd.melt(data.reset_index(), id_vars=['gene_id'],
                      value_vars=['RC.1', 'RC.2', 'RC.3', 'H1.1', 'H1.2', 'H1.3',
                                  'H12.1', 'H12.2', 'H12.3', 'RVN.1', 'RVN.2', 'RVN.3',
                                  'H2.1', 'H2.2', 'H2.3'],
                      var_name='cell_line', value_name='expression')
melted_data

,gene_id,cell_line,expression
0,ENSMUSG00000000001,RC.1,231.612000
1,ENSMUSG00000000028,RC.1,71.304500
2,ENSMUSG00000000031,RC.1,0.000003
3,ENSMUSG00000000049,RC.1,0.060922
4,ENSMUSG00000000056,RC.1,33.078700
...,...,...,...
369625,ENSMUSG00002076463,H2.3,0.106321
369626,ENSMUSG00002076508,H2.3,0.272560
369627,ENSMUSG00002076570,H2.3,0.079742
369628,ENSMUSG00002076650,H2.3,1.148910


In [25]:
# Now filter melted_data for the genes we are interested in
# Ensure that old_gene_list is mapped to Ensembl IDs
target_ensembl_ids = set(old_gene_list)
combined_gene_list = list(target_ensembl_ids) + list(new_gene_list)
# print('combined_gene_list:', type(combined_gene_list), len(combined_gene_list))

# Filter for target genes
filtered_data = melted_data[melted_data['gene_id'].isin(combined_gene_list)]
# print(len(filtered_data))
# (note: filtered_data has 5850 rows bc each gene can appear multiple times bc it appearas in multiple conditions -- 15 cols in og 'data' dataframe * 390 genes = 5850) 
filtered_data['cell_line'] = filtered_data['cell_line'].str.split('.').str[0] # make it so that each cell line replicate has the same label
filtered_data

/var/folders/5h/nw2bw54501q8yzhtr7fv7qnc0000gq/T/ipykernel_39700/540569346.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['cell_line'] = filtered_data['cell_line'].str.split('.').str[0] # make it so that each cell line replicate has the same label


,gene_id,cell_line,expression
21,ENSMUSG00000000159,RC,31.415900
25,ENSMUSG00000000184,RC,0.091381
42,ENSMUSG00000000303,RC,80.686600
53,ENSMUSG00000000365,RC,0.000003
64,ENSMUSG00000000440,RC,0.000003
...,...,...,...
367264,ENSMUSG00000112850,H2,2.985500
368048,ENSMUSG00000115817,H2,0.186060
368419,ENSMUSG00000117406,H2,5.050140
368873,ENSMUSG00000120070,H2,3.388040


In [27]:
# Run ANOVA
for gene_id in filtered_data['gene_id'].unique():
    gene_data = filtered_data[filtered_data['gene_id'] == gene_id]
    lm = ols('expression ~ C(cell_line)', data=gene_data).fit()
    anova_table = sm.stats.anova_lm(lm, typ=2)
    print(f"ANOVA table for gene {gene_id}:\n", anova_table)

    # If the p-value from the ANOVA is significant, proceed with Tukey's HSD
    if anova_table['PR(>F)'][0] < 0.05:  # Assuming index 0 is the p-value for cell_line
        print(f"Performing Tukey's HSD test for gene {gene_id}")
        tukey = pairwise_tukeyhsd(endog=gene_data['expression'],
                                  groups=gene_data['cell_line'],
                                  alpha=0.05)
        print(tukey)



ANOVA table for gene ENSMUSG00000000159:
                    sum_sq    df           F        PR(>F)
C(cell_line)  3627.137481   4.0  383.014574  6.843748e-11
Residual        23.674931  10.0         NaN           NaN
Performing Tukey's HSD test for gene ENSMUSG00000000159
 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
    H1    H12   0.5372 0.9919  -3.5974   4.6719  False
    H1     H2 -32.3484    0.0  -36.483 -28.2138   True
    H1     RC   -2.765 0.2542  -6.8996   1.3696  False
    H1    RVN  -32.455    0.0 -36.5897 -28.3204   True
   H12     H2 -32.8856    0.0 -37.0203  -28.751   True
   H12     RC  -3.3022 0.1377  -7.4369   0.8324  False
   H12    RVN -32.9923    0.0 -37.1269 -28.8576   True
    H2     RC  29.5834    0.0  25.4488   33.718   True
    H2    RVN  -0.1066    1.0  -4.2413    4.028  False
    RC    RVN   -29.69    0.0 -33.8247 -25.5554   True
-------------

In [32]:


id_to_name_dict = pd.Series(gene_conversions['gene_name'].values,index=gene_conversions['gene_id']).to_dict()

# Initialize two dictionaries to hold the ANOVA and Tukey's HSD results
anova_results = {}
tukey_results = {}

# Run ANOVA with a progress bar
for gene_id in tqdm(filtered_data['gene_id'].unique(), desc="Running ANOVA"):
    gene_data = filtered_data[filtered_data['gene_id'] == gene_id]
    if gene_id in id_to_name_dict:
        gene_name = id_to_name_dict.get(gene_id)  
    else:
        gene_name = gene_id
    lm = ols('expression ~ C(cell_line)', data=gene_data).fit()
    anova_table = sm.stats.anova_lm(lm, typ=2)
    anova_results[gene_name] = anova_table  # Store ANOVA result using gene name as key

    # If the p-value from the ANOVA is significant, perform Tukey's HSD
    if anova_table['PR(>F)'][0] > 0.05:
        print(gene_id, 'is non-significant')
        # tukey = pairwise_tukeyhsd(endog=gene_data['expression'],
        #                           groups=gene_data['cell_line'],
        #                           alpha=0.05)
        # tukey_results[gene_name] = tukey.summary()  # Store Tukey's result using gene name as key        

# Now you have two dictionaries with gene names as keys and the results as values


Running ANOVA:  19%|█▉        | 76/390 [00:01<00:07, 41.81it/s]

ENSMUSG00000021302 is non-significant


Running ANOVA:  46%|████▌     | 178/390 [00:06<00:05, 40.85it/s]

ENSMUSG00000028991 is non-significant
ENSMUSG00000029223 is non-significant


Running ANOVA:  53%|█████▎    | 208/390 [00:07<00:04, 40.77it/s]

ENSMUSG00000031004 is non-significant


Running ANOVA:  81%|████████  | 314/390 [00:10<00:02, 34.93it/s]

ENSMUSG00000047126 is non-significant


Running ANOVA: 100%|██████████| 390/390 [00:12<00:00, 30.88it/s]


In [33]:
# Initialize a dictionary to hold the eta-squared results
eta_squared_results = {}

# Run ANOVA and calculate eta-squared
for gene_id in filtered_data['gene_id'].unique():
    if gene_id in id_to_name_dict:
        gene_name = id_to_name_dict.get(gene_id)
    else:
        gene_name = gene_id
    gene_data = filtered_data[filtered_data['gene_id'] == gene_id]
    lm = ols('expression ~ C(cell_line)', data=gene_data).fit()
    anova_table = sm.stats.anova_lm(lm, typ=2)
    ss_factor = anova_table['sum_sq'][0]  # sum_sq for the cell_line factor
    ss_total = anova_table['sum_sq'].sum()  # total sum_sq is the sum of all sum_sq entries
    eta_squared = ss_factor / ss_total
    eta_squared_results[gene_name] = eta_squared  # Store the result


['Ggps1', 'Mtor', 'Uchl1', 'Mki67', 'Cltc']

Cltc from 70 old genes
Ggps1 from 70 old genes
Mki67 from 70 old genes
Uchl1 from 70 old genes
Mtor from 70 old genes


In [ ]:
print(id_to_name_dict)
os.chdir('/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - statin/8. RNA-seq/Elizabeth/LW15 analysis/LW15-Target-Genes/gene_plots')

# Save anova & tukey tests to a CSV file
tukey.to_csv('tukey.csv', index=False)